In [4]:
import pandas as pd
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMRegressor
from sqlalchemy import create_engine




# 파일 불러오기
train_org = pd.read_csv('train_v3.csv')
sample_submission = pd.read_csv("sample_submission.csv")
train_df = train_org.copy()
test_df = sample_submission.copy()



# '일시' 컬럼을 datetime 형식으로 변환하고, Feature을 추출
train_df['일시'] = pd.to_datetime(train_df['일시'])

train_df['년'] = train_df['일시'].dt.year
train_df['월'] = train_df['일시'].dt.month
train_df['일'] = train_df['일시'].dt.day

test_df['일시'] = pd.to_datetime(test_df['일시'])

test_df['년'] = test_df['일시'].dt.year
test_df['월'] = test_df['일시'].dt.month
test_df['일'] = test_df['일시'].dt.day


test_feature = test_df[['년','월','일']]
train_features = train_df[test_feature.columns]
train_target = train_df.drop(columns=['일시', '년','월','일']).copy()
target_columns = train_target.columns


# 교차 검증
kf = KFold(n_splits=4, shuffle=True, random_state=42)


scoring = 'neg_mean_absolute_error' # 어떤 평가지표를 사용할 것인가?

# 모델을 저장할 딕셔너리 생성
models_dict = {}

best_params = {
    "max_depth": 8,
    "n_estimators": 1000,
    "learning_rate": 0.01,
    "random_state": 42,
    "verbose": -1
}

# 각 목표 변수에 대한 모델 학습
for target_col in target_columns:

    # best 모델 인스턴스 생성(정의)
    lgb_model_target = LGBMRegressor(**best_params)

    n_iter = 0
    accuracy_lst = []
    mae_lst = []

    for train_index, valid_index in kf.split(train_features, train_target[target_col]):
        n_iter += 1
        # 학습용, 검증용 데이터 구성
        train_x, valid_x = train_features.iloc[train_index], train_features.iloc[valid_index]
        train_y, valid_y = train_target[target_col].iloc[train_index], train_target[target_col].iloc[valid_index]

        # 학습
        lgb_model_target.fit(train_x, train_y)
        valid_pred = lgb_model_target.predict(valid_x) # 예측값

        # 평가
        mae = mean_absolute_error(valid_y, valid_pred)
        mae_lst.append(mae)
        print(f'{n_iter} 번째 Stratified K-Fold MAE for {target_col}: {mae}')

    # 최종 평가
    print('-'*50)
    print(f'교차 검증 MAE for {target_col}: {np.mean(mae_lst)}')

    # 학습된 모델 저장
    models_dict[target_col] = lgb_model_target

1 번째 Stratified K-Fold MAE for 1: 134.73804717308593
2 번째 Stratified K-Fold MAE for 1: 131.9364321436724
3 번째 Stratified K-Fold MAE for 1: 133.42568301556165
4 번째 Stratified K-Fold MAE for 1: 129.1424392675745
--------------------------------------------------
교차 검증 MAE for 1: 132.3106503999736
1 번째 Stratified K-Fold MAE for 2: 134.63908031531093
2 번째 Stratified K-Fold MAE for 2: 137.3777996292811
3 번째 Stratified K-Fold MAE for 2: 135.16808112889782
4 번째 Stratified K-Fold MAE for 2: 131.02313024341294
--------------------------------------------------
교차 검증 MAE for 2: 134.55202282922568
1 번째 Stratified K-Fold MAE for 3: 136.05266507867643
2 번째 Stratified K-Fold MAE for 3: 133.79953338986402
3 번째 Stratified K-Fold MAE for 3: 136.86440364912812
4 번째 Stratified K-Fold MAE for 3: 135.16207900540599
--------------------------------------------------
교차 검증 MAE for 3: 135.46967028076864
1 번째 Stratified K-Fold MAE for 4: 138.21787807807843
2 번째 Stratified K-Fold MAE for 4: 138.7175032334144
3 